In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy import signal
import xesmf as xe
from keras.models import load_model
 
from mlprecip_utils import *
from mlprecip_models import *
from mlprecip_xai import *
from mlprecip_plot import *

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
/homes/kpegion/.conda/envs/ml-precip/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/homes/kpegion/.conda/envs/ml-precip/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/homes/kpegion/.conda/envs/ml-precip/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/homes/kpegion

In [2]:
varname='precip'
#target_dataset='era5-precip'
target_dataset='era-interim-precip'

model_types=['logmodel_med']
#model_types=['nnmodel_med','logmodel_med']
nmodels=100
model_path='../data/fnn/'+target_dataset+'/models/seus.median'
winter=[12,1,2]
summer=[6,7,8]
cat_labels=['Negative','Positive']

### Read in train and test data

In [3]:
ds_features_tval=xr.open_dataset('../data/fnn/features_trainval.nc')
ds_target_tval=xr.open_dataset('../data/fnn/'+target_dataset+'/target_trainval.nc')
ds_features_test=xr.open_dataset('../data/fnn/features_test.nc')
ds_target_test=xr.open_dataset('../data/fnn/'+target_dataset+'/target_test.nc')

In [4]:
ds_features=[ds_features_tval,ds_features_test]
ds_target=[ds_target_tval,ds_target_test]

### Main Program to Validate Models

In [5]:
for model_type in model_types:
    
    print(model_types)
    # Loop over seasons
    #for seas,slabel,seas_abbrv in zip([winter,summer],['Winter','Summer'],['DJF','JJA']):
    for seas,slabel,seas_abbrv in zip([summer],['Summer'],['JJA']):

        print(slabel)
    
        # Loop over Train-Val and Test
        for (ds_f, ds_t,label) in zip(ds_features, ds_target,['Train-Val','Test']):
        
            print(label)
        
            # Make sure we have the same times for target and features
            ds_t,ds_f=xr.align(ds_t.dropna(dim='time'),
                               ds_f.fillna(0.0),
                               join='inner')
        
            # Select season from target precip anomalies and features
            ds_f_seas=ds_f.sel(time=ds_f['time.month'].isin(seas)) 
            ds_t_seas=ds_t.sel(time=ds_t['time.month'].isin(seas)) 
    
            # Subtract the median to ensure data is centered and classes are equal
            median=np.percentile(ds_t_seas[varname],50)
            ds_t_seas[varname]=ds_t_seas[varname]-median
       
            # Create X Features input
            feature_vars=list(ds_f_seas.keys())
            X=ds_f_seas.to_stacked_array('features',sample_dims=['time']).values

            # One Hot Encode Target (Y)
            Y_ohe=make_ohe_thresh_med(ds_t_seas[varname])
    
            print('Check Features and Target Dimensions')
            print('Features (X): ',X.shape)
            print('Target (Y): ',Y_ohe.shape)
        
            # Loop over Models
            for imodel in range(nmodels):
            
                model_infname=model_path+'.'+seas_abbrv+'.'+model_type+'.'+str(imodel)+'.h5'
                print(model_infname)
                model=load_model(model_infname)
                #model.summary()
            
                # Accuracy Score
                score=model.evaluate(X,Y_ohe) 
                print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
            
                # Predictions
                Yprobs=model.predict(X)
                Ypred=np.argmax(Yprobs, axis = 1)
            
                # Calculate LRP (TO-DO: put this into and return ds_lrp)
                rules=['lrp.alpha_1_beta_0']
                a=calcLRP(model,X.reshape(X.shape[0],X.shape[1]),rules=rules)
                b=np.asarray(a)
                del a
                
                # Put all model output information into a Dataset to be written to a netcdf file 
                ds_lrp=xr.DataArray(b,
                                    coords={'rules':rules,
                                            'time':ds_f_seas['time'],
                                            'var':feature_vars},
                                    dims=['rules','time','var']).to_dataset(name='lrp')    
                del b
        
                ds_pred=xr.DataArray(Ypred,coords={'time':ds_f_seas['time']},
                                     dims=['time']).to_dataset(name='pred')
    
                ds_probs=xr.DataArray(Yprobs,coords={'time':ds_f_seas['time'],
                                                     'cat':cat_labels},
                                            dims=['time','cat']).to_dataset(name='probs')
                ds_acc=xr.DataArray(score[1],
                                    coords={'model':[imodel]},
                                    dims=['model']).to_dataset(name='acc')    
                ds_verif=xr.DataArray(np.argmax(Y_ohe,axis=1),
                                      coords={'time':ds_f_seas['time']},
                                      dims=['time']).to_dataset(name='verif')
          
                ds=xr.merge([ds_lrp,ds_pred,ds_verif,ds_probs,ds_acc])
            
                del ds_lrp,ds_pred,ds_verif,ds_probs,ds_acc
            
                model_ofname='../data/fnn/'+target_dataset+'/validate/model_validate.'+label+'.'+seas_abbrv+'.'+model_type+'.'+str(imodel)+'.nc'
                print(model_ofname)
                ds.to_netcdf(model_ofname)

['logmodel_med']
Summer
Train-Val
Upper Cat:  1748
Lower Cat:  1748
Check Features and Target Dimensions
Features (X):  (3496, 10)
Target (Y):  (3496, 2)
../data/fnn/era-interim-precip/models/seus.median.JJA.logmodel_med.0.h5
3496/3496 [==============================] - 0s 15us/step
acc: 52.92%
../data/fnn/era-interim-precip/validate/model_validate.Train-Val.JJA.logmodel_med.0.nc
../data/fnn/era-interim-precip/models/seus.median.JJA.logmodel_med.1.h5
3496/3496 [==============================] - 0s 19us/step
acc: 48.37%
../data/fnn/era-interim-precip/validate/model_validate.Train-Val.JJA.logmodel_med.1.nc
../data/fnn/era-interim-precip/models/seus.median.JJA.logmodel_med.2.h5
3496/3496 [==============================] - 0s 22us/step
acc: 47.94%
../data/fnn/era-interim-precip/validate/model_validate.Train-Val.JJA.logmodel_med.2.nc
../data/fnn/era-interim-precip/models/seus.median.JJA.logmodel_med.3.h5
3496/3496 [==============================] - 0s 25us/step
acc: 54.26%
../data/fnn/era-i